In [9]:
import torch
from torchvision.models import resnet18, EfficientNet, RegNet
import wandb

In [11]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 13.3 MB/s eta 0:00:00


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, efficientnet_b0, regnet_y_400mf
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from tqdm import tqdm
from torchmetrics.classification import Accuracy

In [2]:
data_dir = '/content/drive/MyDrive/ML_Start/cut_img'

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_data = data_dir + "/train"
test_data = data_dir + "/test"


In [18]:
import torch
from torchvision.models import resnet18, efficientnet_b0, regnet_y_400mf
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from tqdm import tqdm

train_dataset = ImageFolder(train_data, transform=transform)
test_dataset = ImageFolder(test_data, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


def train_and_evaluate(model, model_name):
    wandb.init(
        project=f"{model_name}_project",
        name=model_name,
        config={
            "learning_rate": 0.001,
            "epochs": 3,
            "batch_size": train_loader.batch_size,
        },
    )

    best_accuracy = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    accuracy_metric = Accuracy(task='multiclass',num_classes=2).to(device)

    num_epochs = 3

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()


        model.eval()
        accuracy_metric.reset()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Evaluating"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)
                accuracy_metric.update(preds, labels)
        accuracy = accuracy_metric.compute().item() * 100
        if accuracy > best_accuracy:
          best_accuracy = accuracy
          torch.save(model, f'best_model_{model_name}_{epoch+1}.pth')
        print(f'Epoch №{epoch+1} for model {model_name}')
        print(f"\tTraining Loss: {running_loss/len(train_loader)}\n")
        print(f"\tTest Accuracy: {accuracy:.2f}%\n")
        wandb.log({'loss': running_loss/len(train_loader),
                   'acc': accuracy})
    wandb.finish()

### Обучение модели Resnet

In [17]:
resnet_model = resnet18(pretrained=True)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
train_and_evaluate(resnet_model, "ResNet18")

Epoch 1/3 - Evaluating: 100%|██████████| 11/11 [00:34<00:00,  3.15s/it]


Epoch №1 for model ResNet18
	Training Loss: 0.11125281689289425

	Test Accuracy: 97.70%



Epoch 2/3 - Evaluating: 100%|██████████| 11/11 [00:32<00:00,  2.96s/it]


Epoch №2 for model ResNet18
	Training Loss: 0.047604195939216186

	Test Accuracy: 99.71%



Epoch 3/3 - Evaluating: 100%|██████████| 11/11 [00:33<00:00,  3.09s/it]

Epoch №3 for model ResNet18
	Training Loss: 0.014860138546542398

	Test Accuracy: 100.00%



acc,▁▇█
loss,█▃▁
acc,100
loss,0.01486


### Обучение модели Efficientnet

In [20]:
efficientnet_model = efficientnet_b0(pretrained=True)
num_ftrs = efficientnet_model.classifier[1].in_features
efficientnet_model.classifier[1] = nn.Linear(num_ftrs, len(train_dataset.classes))
train_and_evaluate(efficientnet_model, "EfficientNetB0")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 81.7MB/s]


Epoch 1/3 - Evaluating: 100%|██████████| 11/11 [00:22<00:00,  2.04s/it]


Epoch №1 for model EfficientNetB0
	Training Loss: 0.10493244949750424

	Test Accuracy: 99.71%



Epoch 2/3 - Evaluating: 100%|██████████| 11/11 [00:24<00:00,  2.26s/it]


Epoch №2 for model EfficientNetB0
	Training Loss: 0.025846881739285198

	Test Accuracy: 100.00%



Epoch 3/3 - Evaluating: 100%|██████████| 11/11 [00:25<00:00,  2.29s/it]

Epoch №3 for model EfficientNetB0
	Training Loss: 0.0290064884923465

	Test Accuracy: 99.71%



acc,▁█▁
loss,█▁▁
acc,99.71265
loss,0.02901


### Обучение модели Regnet

In [19]:
regnet_model = regnet_y_400mf(pretrained=True)
num_ftrs = regnet_model.fc.in_features
regnet_model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
train_and_evaluate(regnet_model, "RegNetY_400MF")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_Y_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_Y_400MF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/regnet_y_400mf-c65dace8.pth" to /root/.cache/torch/hub/checkpoints/regnet_y_400mf-c65dace8.pth
100%|██████████| 16.8M/16.8M [00:00<00:00, 50.8MB/s]


Epoch 1/3 - Evaluating: 100%|██████████| 11/11 [00:12<00:00,  1.17s/it]


Epoch №1 for model RegNetY_400MF
	Training Loss: 0.05051019154260367

	Test Accuracy: 100.00%



Epoch 2/3 - Evaluating: 100%|██████████| 11/11 [00:15<00:00,  1.43s/it]


Epoch №2 for model RegNetY_400MF
	Training Loss: 0.01780727593619044

	Test Accuracy: 87.36%



Epoch 3/3 - Evaluating: 100%|██████████| 11/11 [00:14<00:00,  1.34s/it]

Epoch №3 for model RegNetY_400MF
	Training Loss: 0.012617122972317572

	Test Accuracy: 99.71%



acc,█▁█
loss,█▂▁
acc,99.71265
loss,0.01262
